In [8]:
# For, Windows, please import pyvips with this cell
# import os
# vipshome = 'c:\\vips-dev-8.7\\bin'
# os.environ['PATH'] = vipshome + ';' + os.environ['PATH']

In [9]:
import pyvips  #must use conda to install
import os
import glob
from natsort import natsorted
from time import time

In [10]:
def ims2ometiff(ims,outputfilepath):
    imobjs = []
    for idx,im in enumerate(ims):
        impth = os.path.join(input_dir, im)
        # imobj = pyvips.Image.new_from_file(impth)
        imobj = pyvips.Image.openslideload(impth,level=0)
        if imobj.hasalpha(): imobj = imobj[:-1]
        imobjs.append(imobj)
        print('image {} has dimensions [{},{}]'.format(idx,imobj.height,imobj.width))
    print('z stack height :',len(imobjs))
    comp = pyvips.Image.arrayjoin(imobjs, across=1)
    image_height = imobj.height
    image_width = imobj.width
    image_bands = imobj.bands
    comp = comp.copy()

    # set minimal OME metadata
    # before we can modify an image (set metadata in this case), we must take a
    # private copy
    comp.set_type(pyvips.GValue.gint_type, "page-height", image_height)
    comp.set_type(pyvips.GValue.gstr_type, "image-description",
                  f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="{image_bands}"
                    SizeT="1"
                    SizeX="{image_width}"
                    SizeY="{image_height}"
                    SizeZ="{len(imobjs)}"
                    Type="uint8">
            </Pixels>
        </Image>
    </OME>""")


    #jpeg,jp2k,lzw,
    print('saving image...')
    # jp2k breaks the format somehow
    start=time()
    comp.tiffsave(outputfilepath, compression="jpeg", tile=True,
                  tile_width=512, tile_height=512,
                  pyramid=True, subifd=True)
    print('elapsed time: {}'.format(round(time()-start)))
    print('ome-tiff saved here: ',outputfilepath)

In [ ]:
if __name__=='__main__':
    input_dir = '/home/labuser2/Documents/shelter/skin_aging/clue_cohort/wsi'
    output_dir = os.path.join(input_dir, 'zstack')
    if not os.path.exists(output_dir): os.mkdir(output_dir)

    ims = glob.glob(os.path.join(input_dir,'*.ndpi'))
    ims = natsorted(ims)
    ims = ims[:2]
    print(ims)
    ims2ometiff(ims,os.path.join(output_dir,'clue20xstack10.ome.tiff'))


['/home/labuser2/Documents/shelter/skin_aging/clue_cohort/wsi/2022-06-07 13.12.34.ndpi', '/home/labuser2/Documents/shelter/skin_aging/clue_cohort/wsi/2022-06-07 13.18.40.ndpi']
image 0 has dimensions [52096,69120]
image 1 has dimensions [47872,48000]
z stack height : 2
saving image...
